# 先寫一個簡單版的ptt爬蟲

In [6]:
import requests, json
from bs4 import BeautifulSoup

data = []
for page in range(25293, 25294):
    res = requests.get('https://www.ptt.cc/bbs/Gossiping/index{}.html'.format(page), cookies={'over18': '1'})
    soup = BeautifulSoup(res.text)
    
    for j in soup.select('.title a'):
        res = requests.get('https://www.ptt.cc' + j['href'], cookies={'over18': '1'})
        soup = BeautifulSoup(res.text)
        print(j.text)
        data.append({'title':j.text, 'url':'https://www.ptt.cc' + j['href'], 'text':soup.select('#main-content')[0].text})

json.dump(data, open('ptt.json', 'w'))

c:\users\udic\documents\htdocs\publictutorial\venv\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file c:\users\udic\appdata\local\programs\python\python35\Lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


[參選] Kay731
[參選] reinforce15 
[問卦] 為什麼不直接規定店家用可分解塑膠袋？
Re: 如何面對19歲弟弟突然猝逝
[新聞] 綠委爆料 蔡總統優先徵詢陳菊任閣揆
[ＦＢ] 苗博雅:警察開9槍擊斃逃逸外勞
[新聞] 對川普很失望？ 普廷：他又不是我的新娘
[問卦] 台灣自己研發氫彈嗎？
Re: [ＦＢ] 苗博雅FB:中華民國的憲政體制爛到爆炸
[問卦] 日本人學台灣中油玩停電？
[問卦] 我明天要幾點起來
[問卦] 台灣人懶叫小是不是蚵仔太小顆害的？☺
[問卦] 日本男演員的顏值發生了什麼事？
[問卦] 有沒有娘娘腔的金牌殺手？
Re: [問卦] 用什麼形容詞會讓人感覺到有年紀了
[新聞] 普渡完好兄弟　換真人飛撲搶水果
Re: [新聞] 林奕含案件再議 陳國星維持不起訴處分
[問卦] 返校一出來遇到一隻狼 要怎麼空手打?
[問卦] 有調查局的國家
[ＦＢ] 劉宇fb 來酒店


## 困難版

In [13]:
import requests, json
from bs4 import BeautifulSoup

postive_comment=[] 
negative_comment=[] 
for page in range(25293, 25294):
    res = requests.get('https://www.ptt.cc/bbs/Gossiping/index{}.html'.format(page), cookies={'over18': '1'})
    soup = BeautifulSoup(res.text)
    
    for j in soup.select('.title a'):
        res = requests.get('https://www.ptt.cc' + j['href'], cookies={'over18': '1'})
        soup = BeautifulSoup(res.text)
        
        for tag in soup.select(".push"): 
            if (tag.select(".push-tag")[0].text=="推 "): 
                content = str(tag.select(".push-content")[0].text) 
                content = content[2:] 
                postive_comment.append(content) 
            if (tag.select(".push-tag")[0].text=="噓 "): 
                content = str(tag.select(".push-content")[0].text) 
                content = content[2:] 
                negative_comment.append(content) 

print(postive_comment) 
print(negative_comment)

c:\users\udic\documents\htdocs\publictutorial\venv\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file c:\users\udic\appdata\local\programs\python\python35\Lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


['投你了', '推K少', 'YA', '過氣k少又來了', '都沒錢錢發惹', '現在連文摘都沒有 未看先猜精華區也不妙', '不會刪動漫文 戰女文我就投你 嘻嘻', '投你一票。雖然你水桶過我。至少回我文。讓我知道原因', 'K少是少數做的很好了吧 投你了', '老江湖了', '打電話專家', '就決定投你了!                     我來當砲灰的...', '有些版主水桶人，是不知道原因。至少你願意回我文。', '有被你照顧過 我會投你', '讚', 'k可以', 'K少 為什麼要跟Carol0217分手 可以說一下原因嗎', '老屁股保險點 不然又選到怪怪的', 'K少是八卦版唯一會求證爆卦內容的版主。', '發錢喇', '一堆XX在這邊亂 不找清流來整治要找誰?', '死魚我不行', 'ID幫推~', '看過你出來選好幾次了，方方土，還是讓給年輕人吧', 'K少都出來了 沒你的事了', '收費厚塑膠袋的製造工廠受環保署稽查控管，廢掉此制', '推', '好', '可以', '基督教', '謝謝你，我現在腦中真的一片空白，只能用發文的方式紀念他', '質詢到一半睡著嗎?', '宮廷戲?', '別鬧了 派遣之母', '民進黨沒人才，選來選去，不是豬就是瓜', '1陳菊沒有選總統的企圖，對空心菜沒有威脅以權謀的觀', '真他媽噁心', '對總統不滿就實際行動發起罷免 嘴也沒用 還是爽爽任期當', '他有律師牌?  不是只是法律系畢業而已嗎', '先推再看', '現在真的來到多元成家的年代了', '反觀謝龍介', '哈哈哈 小川暴怒', '甲味濃...', '甲上', '普丁攻', '阿謀咧，大家都有，你怎麼可以沒有', '你不用上班哦 真好', '一樓帥哥', '2樓沒懶覺 問他沒用', '同感欸 ..', '五樓真的醜', '土屋太鳳很可愛', '日本的審美跟其他國家不一樣', '我還有點餓', '我不是娘娘腔 我是金牌殺手', '5f娘娘腔包莖殺手', '周大福', '周大福啊', '肩膀有菜花的給我進來', ' 公三小', '西瓜蛇', '人比鬼兇', '人家74你敢嘴？', '不能檢討被害人阿.......... 結果現在是兩情相悅 糗了', '林父母林的電腦日記都不給檢方，真的很莫名', '我就是要罵林父母根本垃圾 有夠鬧劇一場', 